# dargen的使用

In [1]:
%run dargen_package.ipynb

## Define roles and quest

In [2]:
protagonist_name = "Harry Potter"
storyteller_name = "Dungeon Master"
quest = "Find all of Lord Voldemort's seven horcruxes."
word_limit = 50  # word limit for task brainstorming

## Ask an LLM to add detail to the game description

In [3]:
game_description = f"""Here is the topic for a Dungeons & Dragons game: {quest}.
        There is one player in this game: the protagonist, {protagonist_name}.
        The story is narrated by the storyteller, {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of a Dungeons & Dragons player."
)

protagonist_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        Please reply with a creative description of the protagonist, {protagonist_name}, in {word_limit} words or less. 
        Speak directly to {protagonist_name}.
        Do not add anything else."""
    ),
]
llm = ChatOpenAI(temperature=1.0,base_url='https://api.gptsapi.net/v1',api_key=os.environ.get('WildCard_API_KEY'))
protagonist_description = llm(
    protagonist_specifier_prompt
).content

storyteller_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        Please reply with a creative description of the storyteller, {storyteller_name}, in {word_limit} words or less. 
        Speak directly to {storyteller_name}.
        Do not add anything else."""
    ),
]
llm = ChatOpenAI(temperature=1.0,base_url='https://api.gptsapi.net/v1',api_key=os.environ.get('WildCard_API_KEY'))
storyteller_description = llm(
    storyteller_specifier_prompt
).content

/Users/zhaoxuefeng/opt/miniconda3/envs/py310_new/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [11]:
print("Protagonist Description:")
print(protagonist_description)
print("Storyteller Description:")
print(storyteller_description)

Protagonist Description:
Dear Harry, you are a young wizard with untold potential, the chosen one destined to defeat the Dark Lord. You have faced great adversity and have proven yourself a true hero time and time again. Now, with the fate of the wizarding world in your hands, you must embark on a dangerous quest to find and destroy the horcruxes. Are you ready for what lies ahead?
Storyteller Description:
Dungeon Master, you possess the power to bring the world of Harry Potter to life. As the narrator of this adventure, your words shape the characters' destinies. Guide Harry through the trials of finding the horcruxes, and watch as your storytelling transports us to magical lands beyond our imaginings.


## Protagonist and dungeon master system messages

In [5]:
protagonist_system_message = SystemMessage(
    content=(
        f"""{game_description}
Never forget you are the protagonist, {protagonist_name}, and I am the storyteller, {storyteller_name}. 
Your character description is as follows: {protagonist_description}.
You will propose actions you plan to take and I will explain what happens when you take those actions.
Speak in the first person from the perspective of {protagonist_name}.
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of {storyteller_name}.
Do not forget to finish speaking by saying, 'It is your turn, {storyteller_name}.'
Do not add anything else.
Remember you are the protagonist, {protagonist_name}.
Stop speaking the moment you finish speaking from your perspective.
"""
    )
)

storyteller_system_message = SystemMessage(
    content=(
        f"""{game_description}
Never forget you are the storyteller, {storyteller_name}, and I am the protagonist, {protagonist_name}. 
Your character description is as follows: {storyteller_description}.
I will propose actions I plan to take and you will explain what happens when I take those actions.
Speak in the first person from the perspective of {storyteller_name}.
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of {protagonist_name}.
Do not forget to finish speaking by saying, 'It is your turn, {protagonist_name}.'
Do not add anything else.
Remember you are the storyteller, {storyteller_name}.
Stop speaking the moment you finish speaking from your perspective.
"""
    )
)

## Use an LLM to create an elaborate quest description

In [9]:
quest_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(
        content=f"""{game_description}
        
        You are the storyteller, {storyteller_name}.
        Please make the quest more specific. Be creative and imaginative.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the protagonist {protagonist_name}.
        Do not add anything else."""
    ),
]
llm = ChatOpenAI(temperature=1.0,base_url='https://api.gptsapi.net/v1',api_key=os.environ.get('WildCard_API_KEY'))

specified_quest = llm(quest_specifier_prompt).content

print(f"Original quest:\n{quest}\n")
print(f"Detailed quest:\n{specified_quest}\n")

Original quest:
Find all of Lord Voldemort's seven horcruxes.

Detailed quest:
Harry Potter, your task is to find all seven horcruxes of Lord Voldemort and destroy them without being caught by his Death Eaters. You must decipher the clues left by Dumbledore, travel across dangerous terrain, and outsmart Voldemort's traps to complete this deadly mission. May luck and magic be on your side.



## Main Loop

In [6]:
protagonist = DialogueAgent(
    name=protagonist_name,
    system_message=protagonist_system_message,
    model=ChatOpenAI(temperature=0.2,base_url='https://api.gptsapi.net/v1',api_key=os.environ.get('WildCard_API_KEY')),
)
storyteller = DialogueAgent(
    name=storyteller_name,
    system_message=storyteller_system_message,
    model=ChatOpenAI(temperature=0.2,base_url='https://api.gptsapi.net/v1',api_key=os.environ.get('WildCard_API_KEY')),
)

In [7]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = step % len(agents)
    return idx

In [ ]:
max_iters = 6
n = 0

simulator = DialogueSimulator(
    agents=[storyteller, protagonist], selection_function=select_next_speaker
)
simulator.reset()
simulator.inject(storyteller_name, specified_quest)
print(f"({storyteller_name}): {specified_quest}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1